In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from openai import OpenAI, AzureOpenAI
import os
import sys
from dotenv import load_dotenv

load_dotenv()

True

## Promp enginering

Prompt Techniques: 
- Plan-and-Solve Prompting: Improving Zero-Shot Chain-of-Thought Reasoning by Large Language Models: https://arxiv.org/pdf/2305.04091.pdf
- Prompt page: https://www.promptingguide.ai/techniques
- ToT: https://github.com/dave1010/tree-of-thought-prompting
- PAL: https://www.promptingguide.ai/techniques/pal
- CoT: https://www.promptingguide.ai/techniques/cot

## Discussion
- https://github.com/holarissun/PanelGPT

## Chat Completions

In [2]:
sys.path.insert(1, os.path.join( os.getcwd(), ".." ) )
from src.utils.openaiwrapper.openai_wrapper import OpenAIWrapper

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

2025-01-27 20:35:09.321 | INFO     | src.utils.openaiwrapper.openai_wrapper:<module>:17 - Update cost from langchain
2025-01-27 20:35:10.162 | INFO     | src.utils.openaiwrapper.get_openai_cost:extract_and_save_model_cost_data:106 - Model cost data saved to c:\Users\caanp\OneDrive\MBA\Repositorio\GenAI_MBA\src\utils\openaiwrapper\model_costs.json


In [3]:
call_openai = OpenAIWrapper()

### ToT

In [10]:

# ToT: Example

result, usage =call_openai.chat_completion(
    messages=[
        {
            "role": "user",
            "content": """
Imagine three different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.
The question is...            
            
            
```gostei do app, ficou mas bonito, mas não adiante ser bonito se não funciona para nada```
Qual é os sentimento no texto. positivo, negativo ou neutro?

return the result in json format with the sentiment and expert discussion
"""

,
        }
    ],
    model="gpt-4o-mini",
                        )

In [11]:
print(result, "\n\n", usage)

Sure! Here’s how the three experts might discuss the sentiment of the provided text and the outcome in JSON format:

### Expert Discussion Steps

**Expert 1:**
- Step 1: "I notice that the speaker starts with a positive comment about the app, saying they like it and that it looks nicer."
  
**Expert 2:**
- Step 1: "I agree with Expert 1, but I also think that the second part of the statement introduces a negative sentiment, as it emphasizes that looks don't matter if functionality is lacking."

**Expert 3:**
- Step 1: "I see both perspectives. The initial praise suggests a positive sentiment, but the critique on functionality casts a shadow on it, leading me to think the overall sentiment might be negative."

**Expert 1:**
- Step 2: "Based on the initial praise, I might lean towards a mixed sentiment, but the criticism is quite strong, which could indicate a negative overall feeling."

**Expert 2:**
- Step 2: "Yes, the negative part seems to dominate toward the end of the statement. I 

## Panel GPT

- https://github.com/holarissun/PanelGPT

In [251]:
result, usage =call_openai.chat_completion(
    messages=[
        {
            "role": "user",
            "content": """
3 experts are discussing the question with a panel discussion, trying to solve it step by step, and make sure the result is correct and avoid penalty.

The question is...                        
            
```gostei do app, ficou mas bonito, mas não adiante ser bonito se não funciona para nada```
Qual é os sentimento no texto. positivo, negativo ou neutro?

return the result in json format with the sentiment and expert discussion
"""

,
        }
    ],
    model="gpt-3.5-turbo",
                        )

In [252]:
print(result, "\n\n", usage)

{
  "sentiment": "negativo",
  "expert_discussion": [
    {
      "expert": "Linguist Expert",
      "comment": "O texto apresenta uma mistura de sentimentos, elogia a estética do app mas em seguida critica sua funcionalidade. A crítica sobre a falta de funcionalidade é mais forte, resultando em um sentimento negativo geral."
    },
    {
      "expert": "Psychologist Expert",
      "comment": "A ambiguidade do texto mostra uma mistura de sentimentos, com uma ideia positiva inicial seguida por uma avaliação negativa. Isso pode indicar frustração ou decepção."
    },
    {
      "expert": "Tech Expert",
      "comment": "A crítica sobre a falta de funcionalidade do app é um ponto crucial. Mesmo com elogios à estética, a falta de utilidade é um problema grave no contexto da tecnologia, o que leva a um sentimento negativo."
    }
  ]
} 

 {'completion_tokens': 224, 'prompt_tokens': 102, 'total_tokens': 326, 'total_cost': 0.00038700000000000003}


## Funcition Calls + ToT


- Tree of thoughtg: 
    - https://www.promptingguide.ai/techniques/tot
    - https://github.com/dave1010/tree-of-thought-prompting
- Functions call: 
    - https://www.promptingguide.ai/applications/function_calling
    - https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/notebooks/pe-function-calling.ipynb

In [245]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "sentiment_analyzer",
            "description": "Extract sentiments of phrases",
            "parameters": {
                "type": "object",
                "properties": {
                    "sentiment": {
                        "type": "string",
                        "enum":['negative', 'neutral', 'positive'],
                        # "description": "Sentiment in the phrase",
                        "description":"""Imagine three different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave."""
                    },
                    "argumentation": {
                        "type": "string", 
                        "description": "argument",
                        },
                },
                "required": ["sentiment","argumentation"],
            },
        },   
    }
]

In [246]:
call_openai.chat_completion(
    messages=[
        {
            "role": "user",
            "content": """ gostei do app, ficou mas bonito, mas não adiante ser bonito se não funciona para nada """,
        }
    ],
    model="gpt-3.5-turbo",
    tools = tools,
    tool_choice="auto"
                        )

(None,
 {'completion_tokens': 64,
  'prompt_tokens': 149,
  'total_tokens': 213,
  'total_cost': 0.0001705})

In [247]:
rr = call_openai.full_chat_result

In [248]:
dict(rr.choices[0].message)

{'content': None,
 'role': 'assistant',
 'function_call': None,
 'tool_calls': [ChatCompletionMessageToolCall(id='call_x9r5GdQb5XdsDkmMApjVwAHA', function=Function(arguments='{"sentiment":"negative","argumentation":"O usuário gostou da aparência do aplicativo, mas está insatisfeito com a funcionalidade. Ele acredita que a estética não é suficiente se o aplicativo não for funcional."}', name='sentiment_analyzer'), type='function')]}

In [249]:
rr.choices[0].message.tool_calls[0].function.arguments

'{"sentiment":"negative","argumentation":"O usuário gostou da aparência do aplicativo, mas está insatisfeito com a funcionalidade. Ele acredita que a estética não é suficiente se o aplicativo não for funcional."}'

## GPT Instruct

In [7]:
%%time
call_openai.completion(model="gpt-3.5-turbo-instruct", max_tokens =400, prompt="""
Determine o tipo de sentimento, positivo, negativo, neutro do texto a seguir.

```gostei do app, ficou mas bonito, mas não adiante ser bonito se não funciona para nada```

retorne o resultado num json com chaves, sentimento, argumento.
""")

CPU times: total: 0 ns
Wall time: 747 ms


('\n{\n  "sentimento": "negativo",\n  "argumento": "O app não funciona"\n}',
 {'completion_tokens': 23,
  'prompt_tokens': 66,
  'total_tokens': 89,
  'completion_tokens_details': None,
  'prompt_tokens_details': None,
  'total_cost': 0.000145})

# Gemini - OpenAI Key

In [8]:

client = OpenAI(
    api_key=os.getenv("GEMINI_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = client.chat.completions.create(
  model="gemini-1.5-pro",
  messages=[
    {"role": "system", "content": ""},
    {"role": "user", "content": "Hello"}
  ],
  stream=False
)

In [9]:
response

ChatCompletion(id=None, choices=[Choice(finish_reason=None, index=0, logprobs=None, message=ChatCompletionMessage(content='How can I help you today?\n', refusal=None, role='assistant', function_call=None, tool_calls=None), finishReason='stop')], created=1735113412, model='gemini-1.5-pro', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=None, prompt_tokens=None, total_tokens=None, completion_tokens_details=None, prompt_tokens_details=None, completionTokens=8, promptTokens=3, totalTokens=11))

In [3]:
for chunk in response:
    print(chunk.choices[0].delta)

ChoiceDelta(content='Hello', function_call=None, refusal=None, role='assistant', tool_calls=[])
ChoiceDelta(content=' there! How can I help you today?\n', function_call=None, refusal=None, role='assistant', tool_calls=[])


In [11]:
!pip show openai

Name: openai
Version: 1.51.0
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: C:\Users\caanp\OneDrive\MBA\Repositorio\GenAI_MBA\.venv\Lib\site-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: crewai, embedchain, instructor, langchain-openai, litellm, mem0ai, src


In [ ]:
1